In [ ]:
%matplotlib inline
import numpy as np
import tensorflow as tf
import random
import os
import sys
sys.path.append('src/')
import nn
import nibabel as nib
from math import floor, ceil
import pipeline
import Unet
import imgaug as ia
from imgaug import augmenters as iaa

## Augment Single Pair

In [ ]:
# Modify these variables to use the cell below to augment only a single NIfTI pair
nii_data_dir = "" # path to directory where the NIfTI pair is stored
nii_vol_name = "" # filename of volume ending in .nii
nii_seg_name = "" # filename of segmentation ending in .nii

In [ ]:
nii_vol = nib.load(os.path.join(nii_data_dir, nii_vol_name))
nii_seg = nib.load(os.path.join(nii_data_dir, nii_seg_name))

In [ ]:
nii_vol_arr = nii_vol.get_fdata()
nii_seg_arr = nii_seg.get_fdata()

In [ ]:
def elastic_transform_nifti_subs(nifti_vol_arr, nifti_seg_arr, alpha, sigma):
    transformed_vol_arr = np.empty(nifti_vol_arr.shape)
    transformed_seg_arr = np.empty(nifti_seg_arr.shape)
    seed = np.random.randint(0, 2 ** 32 - 1)
    randomizer = np.random.RandomState(seed)
    for i in range(nifti_vol_arr.shape[0]):
        print(i, end='; ')
        rand_arr1 = randomizer.rand(*nifti_vol_arr[i].shape)
        rand_arr2 = randomizer.rand(*nifti_vol_arr[i].shape)
        print("About to transform vol slice;", end=' ')
        transformed_vol_slice = elastic_transform(nifti_vol_arr[i], alpha, sigma, rand_arr1, rand_arr2, int_order=1)
        print("About to transform seg slice;", end=' ')
        transformed_seg_slice = elastic_transform(nifti_seg_arr[i], alpha, sigma, rand_arr1, rand_arr2, int_order=0)
        print("Done with transformations;", end=' ')
        transformed_vol_arr[i] = transformed_vol_slice
        transformed_seg_arr[i] = transformed_seg_slice
        print(' ')
    return transformed_vol_arr, transformed_seg_arr

In [ ]:
alpha = 10
sigma = 3

In [ ]:
transformed_vol_arr, transformed_seg_arr = elastic_transform_nifti_subs(nii_vol_arr, nii_seg_arr, alpha, sigma)

In [ ]:
nii_save_dir = "" # path to directory to place augmented NIfTIs

In [ ]:
pipeline.save_arr_as_nifti(transformed_vol_arr, nii_vol_name, "elastic_def_volume.nii", nii_data_dir, nii_save_dir)
pipeline.save_arr_as_nifti(transformed_seg_arr, nii_seg_name, "elastic_def_seg.nii", nii_data_dir, nii_save_dir)

## Augment All Pairs in Directory

To use this method, all the desired NIfTIs to transform should come from the same subject.

In [ ]:
def elastic_transform_all(source_data_dir, nii_data_dir, nii_save_dir):
    alphas = [5, 10, 15]
    sigmas = [1, 3]
    for folder in os.listdir(source_data_dir):
        files = {}
        for file in os.listdir(os.path.join(source_data_dir, folder)):
            if 'seg' in file:
                files['seg'] = file
            elif 'vol' in file:
                files['volume'] = file
        
        vol_path = os.path.join(os.path.join(source_data_dir, folder), files['volume'])
        nii_vol = nib.load(vol_path)
        nii_vol_arr = nii_vol.get_fdata()
        seg_path = os.path.join(os.path.join(source_data_dir, folder), files['seg'])
        nii_seg = nib.load(seg_path)
        nii_seg_arr = nii_seg.get_fdata()
        
        vol_base_name = files['volume'].split(".nii")[0]
        seg_base_name = files['seg'].split(".nii")[0]
        
        for alpha in alphas:
            for sigma in sigmas:
                transformed_vol_arr, transformed_seg_arr = elastic_transform_nifti_subs(nii_vol_arr, 
                                                                                        nii_seg_arr,
                                                                                        alpha,
                                                                                        sigma)
                new_vol_name = vol_base_name + "_elastic_a" + str(alpha) + "_s" + str(sigma) + ".nii"
                new_seg_name = seg_base_name + "_elastic_a" + str(alpha) + "_s" + str(sigma) + ".nii"
                
                print("Saving seg ", files['seg'], end='; ')
                pipeline.save_arr_as_nifti(transformed_seg_arr, files['seg'], new_seg_name, nii_data_dir, nii_save_dir)
                
                print("Saving volume ", files['volume'], end='; ')
                pipeline.save_arr_as_nifti(transformed_vol_arr, files['volume'], new_vol_name, nii_data_dir, nii_save_dir)

                print("")

In [ ]:
to_deform_dir = "" # path to directory where all the NIfTIs to be augmented are. should contain separate folders for each pair
all_nifti_dir = "" # path to directory with all the NIfTIs associated with the current subject
nii_save_dir  = "" # path to save the augmented NIfTIs

In [ ]:
elastic_transform_all(to_deform_dir, all_nifti_dir, nii_save_dir)